## Lab01: Collect and preprocess data

- Name: ĐOÀN ĐỨC HỮU
- Student code: 21120465


---

## How to do and submit your assignment

**Work on your assignment**

You will do your assignment directly in this notebook. First, fill in your name and ID at the beginning of the file. In the notebook, fill in places that say:

```python
#TODO
```

During your work, you can print out the result, create more cells to test, or create more functions to handle things. Please note that <font color=red>you are not allowed to delete or modify my code cells</font> (except in the case that mentioned above). Let remove `raise NotImplementedError("not implement")` when running code.

Always press `Ctrl + S` in order to save your work.

**Notes:**

    *  Copy means zero
    *  You have to submit your work on time. No exception
    *  Any questions about your grade are based on the assignment submitted on Moodle
    *  Wrong submission takes you -2 points

**Submission guideline**

When grading your assignment, I will choose `Kernel` - `Restart Kernel & Run All Cells` in order to restart the kernel and run all cells in your notebook. Therefore, you should do that before submitting to ensure that the outputs are all as expected.

After that, you make a submited direction as follow:

- Folder `StudentCode` (e.g. If your student code is 1234567, then your folder is `1234567`)
  - File `<StudentCode>.ipynb` (e.g. If your student code is 1234567, then your file is `1234567.ipynb`)

Finally, you compress your folder (`StudentCode`) and submit on Moodle. **The extension of the file is nothing else but `.zip`.**

<font color=red>Please strictly follow the submission rules.</font>


# 1. Set-up environment


In [3]:
#Necessary Packages
import time
import requests
import numpy as np
import pandas as pd 
from bs4 import BeautifulSoup
import re
# If you need other support packages

# 2. Collect data from a website by parsing HTML (3p)


In this section, you are going to collect data from a website simulating the sale of Pokemon. I have prepared all the needed links in a file (`pokemon.txt`), and you have to crawl data from these links. The expected output is a `dataframe` with the following fields:

- `SKU`: ID of Pokemon
- `Name`: Name of Pokemon
- `Price`: Price of Pokemon
- `InStock`: Quantity of Pokemons in stock
- `Categories`: The category of Pokemon
- `Tags`: Tags of Pokemon

Your mission is to complete function `collect_data` with `course_urls_file` as your input parameter. The output should look like `pokemon_example.csv` (I list out some examples so you can easily imagine your work).


In [4]:
pokemon_example = pd.read_csv('pokemon_example.csv')
pokemon_example

,SKU,Name,Price,InStock,Categories,Tags
0,4391,Bulbasaur,63.0,45,"Pokemon, Seed","bulbasaur, Overgrow, Seed"
1,7227,Ivysaur,87.0,142,"Pokemon, Seed","ivysaur, Overgrow, Seed"
2,7036,Venusaur,105.0,30,"Pokemon, Seed","Overgrow, Seed, venusaur"
3,9086,Charmander,48.0,206,"Lizard, Pokemon","Blaze, charmander, Lizard"
4,6565,Charmeleon,165.0,284,"Flame, Pokemon","Blaze, charmeleon, Flame"


In [5]:
def collect_data(course_urls_file):
    #load paths from file
    url_file = open(course_urls_file)
    urls = url_file.readlines()
    urls_filtered = [item[:-1] for item in urls]
    
    #init empty list to store the values of each attribute.
    SKU = []
    Names = []
    Prices = []
    InStocks = []
    Categories = []
    Tags = []

    for url in urls_filtered:
        CLASS_SKU ='sku'
        CLASS_NAME = 'product_title entry-title'
        CLASS_PRICE = 'price'
        CLASS_INSTOCK = 'stock in-stock'
        CLASS_CATEGORY = 'posted_in'
        CLASS_TAG = 'tagged_as'
        r = requests.get(url)
        r.encoding = 'utf-8'
        soup = BeautifulSoup(r.text, 'html.parser')
        # Get SKU
        v = soup.find(name ='span',attrs={'class':CLASS_SKU} )
        sku = v.text if v is not None else str(v)
        SKU.append(sku)

        #Get Name
        v = soup.find(name ='h1',attrs={'class':CLASS_NAME} )
        name = v.text if v is not None else str(v)
        Names.append(name);

        #Get Price
        v = soup.find(name ='p',attrs={'class':CLASS_PRICE} )
        if v is not None:
          price = v.span.text
          price = price.replace('£', '').strip()
          Prices.append(price)
        else:
          Prices.append(None)

        #Get Stock
        v = soup.find(name ='p',attrs={'class':CLASS_INSTOCK} )
        if v is not None:
            stock = v.text
            match = re.search(r'\d+', stock)
            if match:
               stock = match.group()
               InStocks.append(stock)
            else:
                InStocks.append(None)

        #Get categories
        v = soup.find(name='span',attrs={'class':CLASS_CATEGORY})
        if v:
            categories = ', '.join([a.text for a in v.find_all('a')])
            Categories.append(categories)
        else:
            Categories.append(None)

        #Get Tags
        v = soup.find(name='span',attrs={'class':CLASS_TAG})
        if v:
            tags =','.join( [a.text for a in v.find_all('a')])
            Tags.append(tags)
        else:
            Tags.append(None)
        #raise NotImplementedError("not implement")
    
    data = pd.DataFrame({"SKU": SKU,
                         "Name": Names,
                         "Price": Prices, 
                         "InStock": InStocks,
                         "Categories": Categories,
                         "Tags": Tags})
    
    return data

In [ ]:
#TEST
data_pokemon = collect_data("pokemon.txt") 
assert data_pokemon.shape == (755, 6)

In [ ]:
#Save to csv file with name pokemon.csv
data_pokemon.to_csv('pokemon.csv',index = False)
#raise NotImplementedError("not implement")

# 3. Collect data using Web API (4p)


In this section, your work is to practice to crawl data using Web API (http://api.worldbank.org/v2/country/all/indicator/SP.POP.TOTL). This is the data of World Bank which includes demographic data and other statistics related to Population, Employment, Health, GDP, Energy Consumption,... for all countries in the world from 1960 to 2022.

From the following selected indicators:

- `SP.POP.TOTL` - Total population
- `SP.POP.TOTL.FE.IN` - Total female population
- `SP.POP.TOTL.MA.IN` - Total male population
- `SP.DYN.CBRT.IN` - Birth rate
- `SP.DYN.CDRT.IN` - Death rate
- `SP.DYN.LE00.MA.IN` - Average life expectancy of male
- `SP.DYN.LE00.FE.IN` - Average life expectancy of female
- `SE.PRM.ENRR` - Primary school enrollment rate
- `SE.TER.ENRR` - High school enrollment rate
- `SE.PRM.CMPT.ZS` - Primary completion rate
- `SE.ADT.1524.LT.ZS` - Literacy rate of people ages 15-24

You are required to collect data from 7 countries and save to dataframe `data_countries`:

- `US` - United States of America
- `IN` - India
- `CN` - China
- `JP` - Japan
- `CA` - Canada
- `GB` - Great Britain
- `ZA` - South Africa

You can expand your work on collecting data (such as collecting data from other countries and other indicators) by reading: https://datahelpdesk.worldbank.org/knowledgebase/articles/889392-api-documentation

**Hints**:

- Use the based URL: http://api.worldbank.org/v2/
- In order to collect data for each indicator of each country, you can use the URL: "http://api.worldbank.org/v2/countries/{country_code}/indicators/{indicator_code}"
  - `country_code` and `indicator_code` are provided above.
  - For example, you can use the following URL to get the `Total population` of Japan: http://api.worldbank.org/v2/countries/jp/indicators/SP.POP.TOTL


In [ ]:
data_countries_examples = pd.read_csv("countries_example.csv")
data_countries_examples

In [ ]:
BASE_URL = 'http://api.worldbank.org/v2/'
COUNTRIES = ["US", "IN", "CN", "JP", "CA", "GB", "ZA"]
INDICATORS = ['SP.POP.TOTL', 
             'SP.POP.TOTL.FE.IN', 
             'SP.POP.TOTL.MA.IN',
             'SP.DYN.CBRT.IN', 
             'SP.DYN.CDRT.IN',
             'SP.DYN.LE00.MA.IN',
             'SP.DYN.LE00.FE.IN',
             'SE.PRM.ENRR',
             'SE.TER.ENRR',
             'SE.PRM.CMPT.ZS',
             'SE.ADT.1524.LT.ZS']

TITLE = ['Total Population',
         'Female Population',
         'Male Population',
         'Birth Rate',
         'Death Rate',
         'Male Life Expectancy',
         'Female Life Expectancy',
         "School Enrollment, Primary",
         "School Enrollment, Tertiary",
         'Primary completion rate',
         'Literacy Rate',
         'Year'
]
# If you need other initializations

In [ ]:
def collect_data(countryCode, per_page, start_year, end_year):
    params = dict()
    params['format'] = 'json'
    params['per_page'] = per_page
    params['date'] = str(start_year)+':'+str(end_year)
    dataList = []
    for indicator in INDICATORS:
        url = BASE_URL + 'countries/'+countryCode.lower()+'/indicators/'+indicator
        res = requests.get(url,params = params)
        if (res.status_code == 200) and ("message" not in res.json()[0].keys()):
            val = []
            if len(res.json()) > 1:
                for obj in  res.json()[1]:
                    if obj['value'] == "" or obj['value'] == None:
                        val.append(None)
                    else:
                        val.append(float(obj['value']))
            dataList.append(val)
        else:
            print('Error'+str(res.status_code))
            
    dataList.append([year for year in range(end_year,start_year -1 ,-1)])
    df = pd.DataFrame(np.column_stack(dataList),columns = TITLE)
    df['Country'] = countryCode
    return df

    #raise NotImplementedError("not implement")

In [ ]:
def Generate_Countries_Dataset(countryCode_list):
    data = pd.DataFrame()
    for countryCode in countryCode_list:
        data = pd.concat([data, collect_data(countryCode = countryCode, per_page = 100, start_year = 2000, end_year = 2022)], axis=0)
    return data

In [ ]:
#TEST
data_countries = Generate_Countries_Dataset(COUNTRIES)
assert data_countries.shape == (161, 13)

In [ ]:
# Save to csv file with name coutries.csv
data_countries.to_csv('./countries.csv',index = False)
#raise NotImplementedError("not implement")